In [1]:
import pandas as pd
import numpy as np
import csv

import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Data Cleasing
## clean original csv and export to a new cleaned csv

In [2]:
def csv_clean(content):
    content = content.replace(";",",")
    content = content.replace('"','')
    return content

In [3]:
with open("DataSampleA.csv") as f:  #read csv to string
    s = f.read()

clean_s = csv_clean(s)  #clean the string using csv_clean function
result = csv.reader(clean_s.splitlines())  #read string back to _csv.reader object
list_result = list(result) #turn _csv.reader into a list to create a data frame

In [4]:
df = pd.DataFrame(list_result, columns = list_result[0])
df = df.drop(df.index[0])
df = df.reset_index(drop=True)

In [5]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [6]:
for item in df.columns:
    print(item)
    print(df[item].value_counts())

age
34    210
32    198
31    169
36    168
35    162
33    161
38    144
37    137
41    124
30    124
40    123
42    120
39    113
46    105
48    104
43    101
49     99
47     97
45     96
44     95
28     91
27     89
50     88
29     85
55     81
51     81
53     80
58     79
57     78
52     77
56     69
54     66
26     64
59     63
60     43
25     41
24     24
23     19
61     15
66      9
63      7
22      7
64      7
21      6
80      6
62      6
77      6
70      5
75      5
71      5
67      5
73      5
19      4
72      4
65      4
79      4
78      3
74      3
69      3
20      3
76      2
68      2
83      2
86      1
87      1
81      1
Name: age, dtype: int64
job
management       871
blue-collar      829
technician       675
admin.           430
services         360
retired          205
self-employed    161
entrepreneur     150
unemployed       112
housemaid         99
student           74
unknown           33
Name: job, dtype: int64
marital
married     2490
single 

In [7]:
df[df.y == ' ']

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,
1030,38,management,married,tertiary,no,-70,yes,no,cellular,30,jan,140,1,2,3,other,
2774,37,self-employed,married,tertiary,no,0,no,no,unknown,6,jun,143,1,-1,0,unknown,
3949,38,housemaid,married,primary,no,21,no,no,cellular,26,aug,634,2,-1,0,unknown,
3967,48,technician,married,secondary,no,10655,no,no,telephone,31,jul,5,3,-1,0,unknown,


In [8]:
drop_index = df[df.y == ' '].index
df = df.drop(df.index[drop_index])
df = df.reset_index(drop=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994 entries, 0 to 3993
Data columns (total 17 columns):
age          3994 non-null object
job          3994 non-null object
marital      3994 non-null object
education    3994 non-null object
default      3994 non-null object
balance      3994 non-null object
housing      3994 non-null object
loan         3994 non-null object
contact      3994 non-null object
day          3994 non-null object
month        3994 non-null object
duration     3994 non-null object
campaign     3994 non-null object
pdays        3994 non-null object
previous     3994 non-null object
poutcome     3994 non-null object
y            3994 non-null object
dtypes: object(17)
memory usage: 530.6+ KB


In [10]:
df.to_csv("cleaned.csv")

In [11]:
df.isnull().values.any()

False

# Data Prep
## generate success rate for each column then export to csv files

In [13]:
df.y = df.y.apply(lambda x: 1 if x == "yes" else 0)

In [15]:
df = df.rename(columns = {'y':'purchased'})

In [17]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,purchased
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0
4,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,0


In [16]:
df.to_csv("modified.csv")

In [38]:
def success_rate(column, df):
    cate_rate = column + "," + "success_rate"
    for cate in df[column].unique():
        n_called = df[column].where(df[column] == cate).count()
        n_success = df[df[column] == cate].purchased.sum()
        success_rate = n_success / n_called
        cate_rate = cate_rate + "\n" + cate + "," + str(success_rate)
    return cate_rate

In [39]:
success_rate('marital', df)

'marital,success_rate\nmarried,0.09859154929577464\nsingle,0.13946869070208728\ndivorced,0.14505494505494507'

In [41]:
for item in df.columns[:-1]:
    f = open(item + '.csv','w')
    f.write(success_rate(item, df))
    f.close()